In [1]:
import torch
from PIL import Image
import os
import torch.nn.functional as F
import numpy as np

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
input_dir = "/kaggle/input/blood-vessel-segmentation/"
train_dir = input_dir + "train/"
imgs_dir = f"{train_dir}kidney_1_dense/images/"
msks_dir = f"{train_dir}kidney_1_dense/labels/"
slices_ids = sorted(os.listdir(imgs_dir))

In [3]:
size = 10
img = torch.arange(size * size, dtype=torch.float32).view((1, size, size))
img

tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
         [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
         [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
         [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
         [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.],
         [50., 51., 52., 53., 54., 55., 56., 57., 58., 59.],
         [60., 61., 62., 63., 64., 65., 66., 67., 68., 69.],
         [70., 71., 72., 73., 74., 75., 76., 77., 78., 79.],
         [80., 81., 82., 83., 84., 85., 86., 87., 88., 89.],
         [90., 91., 92., 93., 94., 95., 96., 97., 98., 99.]]])

In [4]:
patch_size = 4
overlap = 0

In [5]:
# img = F.pad(img, (0, size % patch_size,
#             0, size % patch_size), mode="replicate")
img

tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
         [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
         [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
         [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
         [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.],
         [50., 51., 52., 53., 54., 55., 56., 57., 58., 59.],
         [60., 61., 62., 63., 64., 65., 66., 67., 68., 69.],
         [70., 71., 72., 73., 74., 75., 76., 77., 78., 79.],
         [80., 81., 82., 83., 84., 85., 86., 87., 88., 89.],
         [90., 91., 92., 93., 94., 95., 96., 97., 98., 99.]]])

In [6]:
unfold = torch.nn.Unfold(kernel_size=(patch_size, patch_size), stride=patch_size - overlap)
fold = torch.nn.Fold(output_size=(size, size), kernel_size=(patch_size, patch_size), stride=patch_size - overlap)
average_mask = 1 / fold(unfold(torch.ones(1, size, size)))
patches = unfold(img).view(patch_size, patch_size, -1).permute(2, 0, 1)
# do prediction
patches = patches.permute(1, 2, 0).view(patch_size * patch_size, -1)
print(patches.shape)

torch.Size([16, 4])


In [7]:
fold(patches) * average_mask

tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7., nan, nan],
         [10., 11., 12., 13., 14., 15., 16., 17., nan, nan],
         [20., 21., 22., 23., 24., 25., 26., 27., nan, nan],
         [30., 31., 32., 33., 34., 35., 36., 37., nan, nan],
         [40., 41., 42., 43., 44., 45., 46., 47., nan, nan],
         [50., 51., 52., 53., 54., 55., 56., 57., nan, nan],
         [60., 61., 62., 63., 64., 65., 66., 67., nan, nan],
         [70., 71., 72., 73., 74., 75., 76., 77., nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]])